In [1]:
import findspark
findspark.init()
# Create SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Aşağıdaki ayarları bilgisayarınızın belleğine göre değiştirebilirsiniz
spark = SparkSession.builder \
.master("local[4]") \
.appName("Dataset-Olusturmak") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

# sparkContext'i kısaltmada tut
sc = spark.sparkContext

In [6]:
veri_dosyasi = "C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\udemy-apache-spark\\data\\omer_seyfettin_forsa_hikaye.txt"

hikaye_df = spark.read.text(veri_dosyasi)

In [8]:
hikaye_df.show(3,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                              

### RDD'de flatMap ile boşluklardan kelimeleri ayırıp aşağı atıyorduk. 
### Yapısal API'de benzer işi sql explode fonksiyonu ile yapıyoruz.

In [21]:
from pyspark.sql.functions import explode, split, col
# Her bir kelimeyi boşluklarla ayıralım ve başka bir rdd'de tutalım
kelimeler = hikaye_df.select(explode(split(col("value"), " ")).alias("value"))
kelimeler.show(5)

+---------+
|    value|
+---------+
|     Ömer|
|Seyfettin|
|        -|
|    Forsa|
|         |
+---------+
only showing top 5 rows



In [22]:
kelimeler.groupBy("value").count().show(10)

+------------+-----+
|       value|count|
+------------+-----+
|    gibiydi.|    3|
|        kırk|    2|
|     iniyor,|    1|
|  Sevinçten,|    1|
|   bilmeyen,|    1|
|  çıkacağım.|    1|
|    minimini|    1|
|        yılı|    2|
|      dikkat|    1|
|yaşındayken,|    1|
+------------+-----+
only showing top 10 rows



In [26]:
from pyspark.sql.functions import desc
kelimeler.groupBy("value").count().orderBy(desc("count")).show(10)

+-----+-----+
|value|count|
+-----+-----+
|     |   59|
|  bir|   33|
|    –|   31|
|  yıl|    8|
| diye|    6|
| Türk|    5|
|dedi.|    5|
| Kırk|    5|
|doğru|    5|
| onun|    5|
+-----+-----+
only showing top 10 rows



## İkinci yöntem

In [29]:
kelimeler.groupBy("value").agg({"value": "count"}).orderBy(desc("count(value)")).show(10)

+-----+------------+
|value|count(value)|
+-----+------------+
|     |          59|
|  bir|          33|
|    –|          31|
|  yıl|           8|
| diye|           6|
| Kırk|           5|
| Türk|           5|
| onun|           5|
|dedi.|           5|
|doğru|           5|
+-----+------------+
only showing top 10 rows



In [31]:
kelimeler.groupBy("value").agg({"value": "count"}).orderBy(desc("count")).show(10)

AnalysisException: "cannot resolve '`count`' given input columns: [count.value, count.count(value)];;\n'Sort ['count DESC NULLS LAST], true\n+- AnalysisBarrier\n      +- SubqueryAlias count\n         +- Aggregate [value#35], [value#35, count(value#35) AS count(value)#128L]\n            +- Project [value#35]\n               +- Generate explode(split(value#2,  )), false, [value#35]\n                  +- Relation[value#2] text\n"